In [1]:
from strategy import *
import pickle
from skimage.metrics import structural_similarity as ssim

In [2]:
no_reg = []
reg = []
zero_prior = []
gen_tikhonov = []
non_negative_ls = []
nnls_tikhonov = []

In [3]:
def write_value(value):
    with open('maps.pkl', 'wb') as f:
        pickle.dump(value, f)


def read_value():
    with open('maps.pkl', 'rb') as f:
        val = pickle.load(f)
    return val


def calculate_ssim(img1, img2):
    img1 = np.clip(np.array(img1)*255, 0, 255).astype(np.uint8)
    img2 = np.clip(np.array(img2)*255, 0, 255).astype(np.uint8)
    return ssim(img1, img2)


def calculate_mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err


def calculate_psnr(img1, img2):
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

In [4]:
SIZE = 10
NUM_MAPS = 8
ITERATION = 1000
SPARSITY = 0.6
maps = {}
for i in range(NUM_MAPS):
    true, pred, A, B = strategy1(SPARSITY, i, SIZE, ITERATION)
    no_reg.append((true, pred))
    true, pred, A, B = strategy2(SPARSITY, i, SIZE, ITERATION)
    reg.append((true, pred))
    true, pred, A, B = strategy2(SPARSITY, i, SIZE, ITERATION, True)
    zero_prior.append((true, pred))
    true, pred, A, B = strategy3(SPARSITY, i, SIZE, ITERATION)
    gen_tikhonov.append((true, pred))
    true, pred, A, B = strategy4(SPARSITY, i, SIZE, ITERATION)
    non_negative_ls.append((true, pred))
    true, pred, A, B = strategy5(SPARSITY, i, SIZE, ITERATION)
    nnls_tikhonov.append((true, pred))
    plt.close()
        # break

100%|██████████| 1000/1000 [00:10<00:00, 98.65it/s]


In [6]:
psnr_no_reg = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), no_reg))
ssim_no_reg = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), no_reg))
mse_no_reg = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), no_reg))

psnr_reg = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), reg))
ssim_reg = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), reg))
mse_reg = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), reg))

psnr_zero_prior = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), zero_prior))
ssim_zero_prior = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), zero_prior))
mse_zero_prior = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), zero_prior))

psnr_gen_tikhonov = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), gen_tikhonov))
ssim_gen_tikhonov = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), gen_tikhonov))
mse_gen_tikhonov = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), gen_tikhonov))

psnr_non_negative_ls = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), non_negative_ls))
ssim_non_negative_ls = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), non_negative_ls))
mse_non_negative_ls = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), non_negative_ls))

psnr_nnls_tikhonov = list(
    map(lambda imgs: calculate_psnr(imgs[0], imgs[1]), nnls_tikhonov))
ssim_nnls_tikhonov = list(
    map(lambda imgs: calculate_ssim(imgs[0], imgs[1]), nnls_tikhonov))
mse_nnls_tikhonov = list(
    map(lambda imgs: calculate_mse(imgs[0], imgs[1]), nnls_tikhonov))

fig, ax = plt.subplots(1, 3, figsize=(17, 5))
ax[0].plot(psnr_no_reg, label='without reg.')
ax[0].plot(psnr_reg, label='with reg.')
ax[0].plot(psnr_zero_prior, label='zero prior')
ax[0].plot(psnr_gen_tikhonov, label='gen_tikhonov')
ax[0].plot(psnr_non_negative_ls, label='non_negative_ls')
ax[0].plot(psnr_nnls_tikhonov, label='nnls_tikhonov')
ax[0].set_xlabel('ith image')
ax[0].set_ylabel('PSNR value')
ax[0].legend()

ax[1].plot(mse_no_reg, label='without reg.')
ax[1].plot(mse_reg, label='with reg.')
ax[1].plot(mse_zero_prior, label='zero prior')
ax[1].plot(mse_gen_tikhonov, label='gen_tikhonov')
ax[1].plot(mse_non_negative_ls, label='non_negative_ls')
ax[1].plot(mse_nnls_tikhonov, label='nnls_tikhonov')
ax[1].set_xlabel('ith image')
ax[1].set_ylabel('MSE value')
ax[1].legend()

ax[2].plot(ssim_no_reg, label='without reg.')
ax[2].plot(ssim_reg, label='with reg.')
ax[2].plot(ssim_zero_prior, label='zero prior')
ax[2].plot(ssim_gen_tikhonov, label='gen_tikhonov')
ax[2].plot(ssim_non_negative_ls, label='non_negative_ls')
ax[2].plot(ssim_nnls_tikhonov, label='nnls_tikhonov')
ax[2].set_xlabel('ith image')
ax[2].set_ylabel('SSIM value')
ax[2].legend()

plt.savefig('plots/metrics.png', bbox_inches='tight')
plt.close()

maps['no_reg'] = no_reg
maps['reg'] = reg
maps['zero_prior'] = zero_prior
maps['gen_tikhonov'] = gen_tikhonov
maps['non_negative_ls'] = non_negative_ls
maps['nnls_tikhonov'] = nnls_tikhonov
write_value(maps)


In [8]:
ssim_no_reg

[0.9771819530025431,
 0.7713559650183872,
 0.9378362690356021,
 0.8826949080990265,
 0.863885481135727,
 0.8301013484077535,
 0.9321412273903837,
 0.8679953683321873]